<a href="https://colab.research.google.com/github/mainarel/Numerical_methods_7/blob/main/Lab3_minimal_residual_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse import csr_matrix

Заполнение матрицы, границы области, шаг сетки:

In [7]:
n = 5

A = np.random.rand(n, n) * 10

print('Матрица А: \n', A)

B = np.random.rand(n, 1) * 10
print('Матрица B: \n', B)

AM = np.column_stack((A, B[:, 0]))

print('Расширенная матрица А: \n', AM)

x_min = 0
x_max = 1
y_min = 0
y_max = 2

Матрица А: 
 [[5.01864779 8.05969617 2.66070279 4.60952357 2.38147712]
 [4.19064345 5.97705647 1.84088782 9.88095509 4.47319635]
 [2.38692906 8.77694391 2.09528765 1.57008879 1.94392997]
 [1.29963947 2.36935616 2.73697653 6.90149551 3.25346654]
 [9.51977353 3.54137629 4.1133648  2.44061299 2.61941923]]
Матрица B: 
 [[2.9254699 ]
 [0.74628697]
 [0.76417813]
 [3.42591578]
 [4.61323659]]
Расширенная матрица А: 
 [[5.01864779 8.05969617 2.66070279 4.60952357 2.38147712 2.9254699 ]
 [4.19064345 5.97705647 1.84088782 9.88095509 4.47319635 0.74628697]
 [2.38692906 8.77694391 2.09528765 1.57008879 1.94392997 0.76417813]
 [1.29963947 2.36935616 2.73697653 6.90149551 3.25346654 3.42591578]
 [9.51977353 3.54137629 4.1133648  2.44061299 2.61941923 4.61323659]]


**Условия задачи:** \\
*Используя метод минимальных невязок найти решение СЛАУ, возникающей при разностной аппроксимации задачи Дирихле для двумерного уравнения Пуассона в прямоугольнике.
Даны точные решения.* \\
$$ u(x,y) = 5(-3x+4y)cos(5x+2y)+3y^2 \\
0 \leq x \leq 1, \\ 0 \leq y \leq 2 
$$
*Вычислим производные:*
$$
u_{xx} = 150sin(5x+2y) - 125cos(5x+2y)(-3x+4y) \\
u_{yy} = -80sin(5x+2y)-20cos(5x+2y)(-3x+4y)+6 \\
f(x,y)=u_{xx} + u_{yy}  = 70sin(5x+2y)-145cos(5x+2y)(-3x+4y)+6
$$

*Ограничения:*
$$
\begin{cases}
u(0, y)=\mu(0, y)=\mu_{x}^{(1)}(y) = 20y cos(2y)+3y^2
\\
u(1, y)=\mu(1, y)=\mu_{x}^{(2)}(y) = (-15+20y)cos(5+2y)+3y^2
\\
u(x, 0)=\mu(x, 0)=\mu_{y}^{(1)}(x) = -15xcos(5x)
\\
u(x, 2)=\mu(x, 2)=\mu_{y}^{(2)}(x) = (-15x+40)cos(5x+4)+12
\end{cases}
$$

Граничные условия, аналитическое решение:


In [8]:
def f(x,y):
  return 70*np.sin(5*x+2*y)-145*np.cos(5*x+2*y)*(-3*x+4*y)+6

def u(x,y):
  return 5*(-3*x+4*y)*np.cos(5*x+2*y)+3*y**2
    
def mu1_x(y):
    return 20*y*np.cos(2*y)+2*y**2

def mu2_x(y):
    return (-15+20*y)*np.cos(5+2*y)+3*y**2

def mu1_y(x):
    return -15*x*np.cos(5*x)

def mu2_y(x):
    return (-15*x+40)*np.cos(5*x+4)+12


Дополнительные функции:

In [13]:
#перестановка строк
def swap(matrix, index_from_line, index_to_line):
  line1, line2 = np.copy(matrix[index_from_line, :]), np.copy(matrix[index_to_line, :])
  matrix[index_to_line, :] = line1
  matrix[index_from_line, :] = line2
  return matrix

#число обусловленности
def condition_number(A):  
    return np.linalg.cond(A)

#ошибка
def error(array):
  real_array = np.linalg.solve(A, B)
  return np.linalg.norm(real_array-array)

#невязка
def delta(array):
  return np.linalg.norm(np.dot(A, array) - B.T)

#деление строки на число
def line_division(matrix, line_number, koef):
        matrix[line_number] = [element / koef if element != 0 else 0 for element in matrix[line_number]]

# Совмещение строк
def line_merging(matrix, index_from_line, index_to_line, multiplier):
       matrix[index_to_line] += multiplier*np.array(matrix[index_from_line])


Метод наименьших невязок:

In [15]:
def min_residual_method(A,B):
  

[[ 5.01864779  8.05969617  2.66070279  4.60952357  2.38147712]
 [29.28388238 46.2755373  15.14440175 32.92857294 16.38058193]
 [ 2.38692906  8.77694391  2.09528765  1.57008879  1.94392997]
 [ 1.29963947  2.36935616  2.73697653  6.90149551  3.25346654]
 [ 9.51977353  3.54137629  4.1133648   2.44061299  2.61941923]]
None
